In [1]:
import pyMRAW
import os
import cv2
from tqdm import tqdm 

In [2]:
path=os.path.join(os.path.expanduser("~"), "Desktop", "Side_BI_32mm") #Path od the original cihx file

# Create output directory if it doesn't exist
output_dir = os.path.join(os.path.expanduser("~"), "Desktop", "img_video")
os.makedirs(output_dir, exist_ok=True)

In [3]:
images, info = pyMRAW.load_video(path+'/Side_Video_BI.cihx')

/opt/anaconda3/envs/runpod/lib/python3.12/site-packages/pyMRAW.py:104: UserWarning: Clipped footage! (Total frame: 13214, Original total frame: 43666)
  warnings.warn('Clipped footage! (Total frame: {}, Original total frame: {})'.format(cih['Total Frame'], cih['Original Total Frame'] ))
/opt/anaconda3/envs/runpod/lib/python3.12/site-packages/pyMRAW.py:128: UserWarning: 12bit images will be loaded into memory!
  warnings.warn("12bit images will be loaded into memory!")


In [4]:
info

{'Date': '2023/8/25',
 'Camera Type': 'FASTCAM Nova S16 type 1100K-M-64GB 10GbE model',
 'Record Rate(fps)': 16000.0,
 'Shutter Speed(s)': 60000.0,
 'Total Frame': 13214,
 'Original Total Frame': 43666,
 'Image Width': 1024,
 'Image Height': 1024,
 'File Format': 'Mraw',
 'EffectiveBit Depth': 12,
 'EffectiveBit Side': 'Lower',
 'Color Bit': 12,
 'Comment Text': ''}

In [5]:
images[0].shape #Shape of one frame

(1024, 1024)

In [ ]:
# Save each image as a separate file - Start from 7199 for the above file since before mostly empty
for i, img in tqdm(enumerate(images), total=len(images), desc="Saving frames"):
    # Apply contrast enhancement
    # The image is 12-bit, so we need to normalize it properly to 8-bit
    # Scale from 12-bit (0-4095) to 8-bit (0-255)
    # img_normalized = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX) #Doesn't make any difference
    img_32bit = img.astype('float32')#'uint8')

    # Apply histogram equalization to enhance contrast
    # img_8bit = cv2.equalizeHist(img_8bit) #CLAHE, GaussianBlur, Canny, not working

    alpha = 2  # Contrast (1.0-3.0)
    beta = 30    # Brightness (0-100) - Lower might be better for bubbles?
    img_adjusted = cv2.convertScaleAbs(img_32bit, alpha=alpha, beta=beta) #Makes it a bit lighter

    # Rotate image 90 degrees clockwise
    img_rotated = cv2.rotate(img_adjusted, cv2.ROTATE_90_CLOCKWISE)
    output_path = os.path.join(output_dir, f'frame_{i:05d}.png')
    cv2.imwrite(output_path, img_rotated)

print(f"Saved {len(images)} frames to {output_dir}")

Save every 10 frames to video

In [ ]:
# Get list of frame files
frame_files = sorted([f for f in os.listdir(output_dir) if f.startswith('frame_')])

# Read first frame to get dimensions
first_frame = cv2.imread(os.path.join(output_dir, frame_files[0]))
height, width = first_frame.shape[:2]

# Create video writer
fps = 30  # You can adjust this value
output_video = os.path.join(output_dir, 'output_video.mp4')
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

# Write frames to video
print("Creating video...")
for i, frame_file in enumerate(tqdm(frame_files, desc="Processing frames")):
    if i % 10 == 0:  # Only process every 10th frame
        frame = cv2.imread(os.path.join(output_dir, frame_file))
        out.write(frame)

# Release video writer
out.release()
print(f"Video saved to {output_video}")